In [1]:
from cvrp_parser import parse_cvrplib_uchoa_2014

In [2]:
# read the file Uchoa_et_al_2014/X-n101-k25.vrp
file_path = 'Uchoa_et_al_2014/X-n670-k130.vrp'
#load the text into a string
with open(file_path, 'r') as file:
    vrp_text = file.read()
# parse the CVRP data
inst  = parse_cvrplib_uchoa_2014(vrp_text)

In [3]:
print(inst.name, inst.n_customers, inst.capacity, inst.edge_weight_type)
print("Depot coords:", inst.coords[0])
print("First customer coords:", inst.coords[1])
print("Distance(0,1):", inst.dist(0,1))

X-n670-k130 669 129 EUC_2D
Depot coords: [612. 199.]
First customer coords: [864. 279.]
Distance(0,1): 264.39364591457183


In [4]:
from constructors import clarke_wright_parallel
from utils import build_distance_matrix, is_feasible
from plot_utils import save_solution, plot_solution_from_file

# Clarke–Wright
cw = clarke_wright_parallel(inst.coords, inst.demand, inst.capacity, inst.edge_weight_type)
print("[CW] routes:", len(cw.routes), "cost:", cw.cost,
      "feasible:", is_feasible(cw.routes, inst.demand, inst.capacity, must_cover_all=True, n_customers=inst.n_customers))

# 1) Save to .sol
sol_path = save_solution(inst, cw.routes, cw.cost)  # -> solutions/X-n101-k25/X-n101-k25.sol

# 2) Plot from .sol
png_path = plot_solution_from_file(inst, sol_path)    # -> solutions/X-n101-k25/X-n101-k25.png
print("Wrote:", sol_path, "and", png_path)

[CW] routes: 147 cost: 158715.27300808093 feasible: True
Wrote: solutions\X-n670-k130\X-n670-k130.sol and solutions\X-n670-k130\X-n670-k130.png


In [8]:
from local_search import local_search
# Local search
# Use verbose=True for quick logging, or pass your own logger
ls = local_search(
    coords=inst.coords,
    demand=inst.demand,
    Q=inst.capacity,
    routes=[r[:] for r in cw.routes],
    edge_weight_type=inst.edge_weight_type,
    verbose=True,
    time_limit_sec=60.0,   # stop early, keep best
    improvement_eps=1e-4
)

print(f"BEST cost: {ls.cost:.2f}, routes: {len(ls.routes)}")
#write the LS solution
sol_path_ls = save_solution(inst, ls.routes, ls.cost)
png_path_ls = plot_solution_from_file(inst, sol_path_ls)
print("Wrote:", sol_path_ls, "and", png_path_ls)

17:33:44 [INFO] LS start | routes=147 cost=158715.2730 | time_limit=60.0s | no_gain_passes=2 | impr_eps=0.0001
17:33:44 [INFO]   -> new BEST after 2-opt r=17 i=2 k=5 | best=158712.3730
17:33:44 [INFO] [2-opt] r=17 i=2 k=5 kept
17:33:44 [INFO]   -> new BEST after 2-opt r=17 i=3 k=4 | best=158712.1978
17:33:44 [INFO] [2-opt] r=17 i=3 k=4 kept
17:33:44 [INFO]   -> new BEST after 2-opt r=17 i=3 k=5 | best=158696.8454
17:33:44 [INFO] [2-opt] r=17 i=3 k=5 kept
17:33:44 [INFO]   -> new BEST after 2-opt r=55 i=1 k=2 | best=158675.2049
17:33:44 [INFO] [2-opt] r=55 i=1 k=2 kept
17:33:44 [INFO]   -> new BEST after 2-opt r=83 i=4 k=5 | best=158671.8513
17:33:44 [INFO] [2-opt] r=83 i=4 k=5 kept
17:33:44 [INFO]   -> new BEST after 2-opt r=83 i=3 k=4 | best=158655.2915
17:33:44 [INFO] [2-opt] r=83 i=3 k=4 kept
17:33:44 [INFO]   -> new BEST after 2-opt r=88 i=1 k=4 | best=158650.6898
17:33:44 [INFO] [2-opt] r=88 i=1 k=4 kept
17:33:44 [INFO]   -> new BEST after 2-opt r=88 i=2 k=4 | best=158644.6765
17:

BEST cost: 158464.40, routes: 147
Wrote: solutions\X-n670-k130\X-n670-k130.sol and solutions\X-n670-k130\X-n670-k130.png


In [6]:
# from ils import run_ils, ILSConfig
# cfg = ILSConfig(
#     time_limit_sec=70.0,
#     ls_time_slice_sec=2.0,
#     max_passes_without_gain=2,
#     k_nearest=20,
#     T_init_factor=0.01,
#     T_cooling=0.98,
#     seed=0,
#     verbose=True,
# )
# ils_out = run_ils(
#     coords=inst.coords,
#     demand=inst.demand,
#     Q=inst.capacity,
#     routes_initial=[r[:] for r in cw.routes],
#     edge_weight_type=inst.edge_weight_type,
#     round_euclidean=False,
#     cfg=cfg,
# )

# print(f"[ILS] best_cost={ils_out.cost:.2f} routes={len(ils_out.routes)} in {ils_out.elapsed_sec:.1f}s (iters={ils_out.iterations})")

# # 3) Check, save, plot
# ok = is_feasible(ils_out.routes, inst.demand, inst.capacity, must_cover_all=True, n_customers=inst.n_customers)
# print("Feasible:", ok)

# sol_path = save_solution(inst, ils_out.routes, ils_out.cost)
# png_path = plot_solution_from_file(inst, sol_path)
# print("Wrote:", sol_path, "and", png_path)

In [9]:
from tabu_search import tabu_search

ts = tabu_search(
    coords=inst.coords,
    demand=inst.demand,
    Q=inst.capacity,
    routes_init=[r[:] for r in cw.routes],
    edge_weight_type=inst.edge_weight_type,
    time_limit_sec=70.0,             # hard wall
    max_iters=100000,
    max_no_improve=500,              # patience
    tabu_tenure=15,                  # or tabu_tenure_range=(12,20)
    aspiration=True,
    k_nearest=20,
    max_intra_candidates_per_route=64,
    max_inter_candidates_per_route=128,
    verbose=True,
    seed=42,
)

print(f"[TS] routes={len(ts.routes)} cost={ts.cost:.2f}")

# Save & plot
sol_path = save_solution(inst, ts.routes, ts.cost)
png_path = plot_solution_from_file(inst, sol_path)
print("Wrote:", sol_path, "and", png_path)

17:34:07 [INFO] TS start | routes=147 cost=158715.2730 | time_limit=70.0s | tenure=15 | patience=500
17:34:08 [INFO] [ITER 1] swap_inter kept | Δreal=-32.8690 | curr=158682.4040 | best=158715.2730
17:34:08 [INFO]   -> new BEST at iter 1 | best=158682.4040 (swap_inter)
17:34:09 [INFO] [ITER 2] swap_inter kept | Δreal=-28.9076 | curr=158653.4964 | best=158682.4040
17:34:09 [INFO]   -> new BEST at iter 2 | best=158653.4964 (swap_inter)
17:34:10 [INFO] [ITER 3] reloc_intra kept | Δreal=-19.9134 | curr=158633.5830 | best=158653.4964
17:34:10 [INFO]   -> new BEST at iter 3 | best=158633.5830 (reloc_intra)
17:34:11 [INFO] [ITER 4] reloc_intra kept | Δreal=-15.0454 | curr=158618.5377 | best=158633.5830
17:34:11 [INFO]   -> new BEST at iter 4 | best=158618.5377 (reloc_intra)
17:34:12 [INFO] [ITER 5] reloc_intra kept | Δreal=-13.2956 | curr=158605.2420 | best=158618.5377
17:34:12 [INFO]   -> new BEST at iter 5 | best=158605.2420 (reloc_intra)
17:34:14 [INFO] [ITER 6] reloc_intra kept | Δreal=-12

[TS] routes=147 cost=158463.89
Wrote: solutions\X-n670-k130\X-n670-k130.sol and solutions\X-n670-k130\X-n670-k130.png
